# Tutorial 7: Introduction to LangGraph

In this tutorial, we'll explore LangGraph, a powerful library for building complex, stateful workflows with language models.

## 1. What is LangGraph and how does it differ from LangChain?

LangGraph is an extension of LangChain that focuses on building stateful, multi-step workflows. While LangChain provides building blocks for working with language models, LangGraph allows you to create more complex, graph-based workflows with explicit state management.

Key differences:
- LangGraph introduces the concept of a 'graph' to represent workflows
- It provides better support for stateful applications
- LangGraph allows for more complex branching and conditional logic
- It's designed for building multi-agent systems and complex decision-making processes

## 2. Basic concepts: Nodes, Edges, and Graphs

In LangGraph:
- **Nodes** represent individual steps or actions in a workflow
- **Edges** define the connections and flow between nodes
- **Graphs** combine nodes and edges to create a complete workflow

These concepts allow you to create more structured and complex applications compared to linear chains in LangChain.

## 3. Setting up LangGraph

Let's start by importing the necessary modules and setting up our environment:

In [7]:
!pip install -U langgraph langchain_core




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 3.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.2.41
    Uninstalling langchain-core-0.2.41:
      Successfully uninstalled langchain-core-0.2.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.5 requires langchain-core<0.3,>=0.1.45, but you have langchain-core 0.3.12 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.12 which is incompatible.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.12 which is incompatible.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.3.12 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but

In [6]:
import os
from typing import Dict, TypedDict
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseMessage
from langgraph.graph import StateGraph, END

# Set up the Groq LLM
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0.1,
        model_kwargs={"top_p": 0.5, "seed": 1337}
    )
# Define our state structure
class State(TypedDict):
    messages: List[BaseMessage]
    next_step: str

ImportError: cannot import name 'RemoveMessage' from 'langchain_core.messages' (/home/domenico/miniconda3/lib/python3.12/site-packages/langchain_core/messages/__init__.py)

## 4. Creating your first LangGraph flow

Let's create a simple conversational agent using LangGraph. This agent will greet the user, ask for their name, and then provide a personalized response.

In [ ]:
# Define our graph
workflow = StateGraph(State)

# Define our nodes
def greet(state: State) -> State:
    state["messages"].append({"role": "assistant", "content": "Hello! What's your name?"})
    state["next_step"] = "get_name"
    return state

def get_name(state: State) -> State:
    name = state["messages"][-1]["content"]
    state["messages"].append({"role": "assistant", "content": f"Nice to meet you, {name}! How can I assist you today?"})
    state["next_step"] = "end"
    return state

# Add nodes to the graph
workflow.add_node("greet", greet)
workflow.add_node("get_name", get_name)

# Define edges
workflow.set_entry_point("greet")
workflow.add_edge("greet", "get_name")
workflow.add_edge("get_name", END)

# Compile the graph
app = workflow.compile()

Now that we have created our simple LangGraph flow, let's test it:

In [ ]:
# Initialize the state
initial_state = {"messages": [], "next_step": ""}

# Run the graph
for output in app.stream(initial_state):
    if "messages" in output:
        for message in output["messages"]:
            if message["role"] == "assistant":
                print(f"Assistant: {message['content']}")
            elif message["role"] == "human":
                print(f"Human: {message['content']}")
    
    if output["next_step"] == "get_name":
        user_input = input("Your response: ")
        output["messages"].append({"role": "human", "content": user_input})

print("Conversation ended.")

## Conclusion

In this tutorial, we've introduced LangGraph and created a simple conversational flow. We've covered the basic concepts of nodes, edges, and graphs, and demonstrated how to set up a LangGraph workflow.

This is just the beginning of what you can do with LangGraph. In the next tutorial, we'll explore more complex flows, including branching logic and integration with LangChain components.